# Data Analyst Technical Challenge

# Scenario: Analysing Medical Claims Data for Cost Reduction Opportunities

Importing the necessary libraries

In [1]:
pip install prettytable

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
import pandas as pd
import re
from datetime import datetime
import matplotlib.pyplot as plt
from prettytable import PrettyTable

In [3]:
# Lets first import data from csv files
demography=pd.read_csv('demography.csv')
claim=pd.read_csv('claim.csv')
demography_claim_df=pd.read_csv('demography_claim.csv') #this is just merged member demography and claims

In [4]:
#Exploring members data at a glance
demography

,id,createdAt,updatedAt,firstName,lastName,gender,yearOfBirth,location,facilityName
0,9b2af029-7584-4229-8710-7335806c1440,2024-04-23 00:50:51.717305,2024-04-23 00:50:51.717305,Carolyn,Fritsch,FEMALE,1973,Lake Newell,Forttown
1,7feb5992-93e6-4b12-bf82-098e10f5291d,2024-04-23 00:50:51.879266,2024-04-23 00:50:51.879266,Fred,Heller,FEMALE,1982,Valerieworth,La Stamal
2,12c23628-57ab-414b-a2d0-15ce6f756db2,2024-04-23 00:50:52.001138,2024-04-23 00:50:52.001138,Ella,Rath,FEMALE,1913,Louton,Grand Mountnell
3,3bb1ebe6-583b-4abd-a764-fe765383814d,2024-04-23 00:50:52.116767,2024-04-23 00:50:52.116767,Erik,Jaskolski,FEMALE,1922,Lessiestad,Prince Statenor
4,9f4e6e70-8a2a-48b5-ba7d-b333a3174524,2024-04-23 00:50:52.244694,2024-04-23 00:50:52.244694,Flora,Crona,MALE,1917,Berwyn,Great Ponar
...,...,...,...,...,...,...,...,...,...
38742,fc9a6762-cd76-452f-b324-c17a05108edc,2024-04-23 01:11:55.208897,2024-04-23 01:11:55.208897,Christie,Goyette,FEMALE,2005,Lake Davionhaven,Saint Entian
38743,1e03a6c5-adde-451f-b22b-ae04d21205d5,2024-04-23 01:11:55.238052,2024-04-23 01:11:55.238052,Enrique,McCullough-Littel,MALE,1967,Hoseaport,Lumbsur Ferry
38744,a9ffcb5a-36b3-4d7d-a3a9-8666732de05c,2024-04-23 01:11:55.268909,2024-04-23 01:11:55.268909,Annie,Mosciski,MALE,1952,South Marcohaven,Ftushill
38745,b1dcd835-305f-43dc-be5c-75f62d7cf04e,2024-04-23 01:11:55.299543,2024-04-23 01:11:55.299543,Nellie,Frami,FEMALE,1983,Lake Casimir,Lynstell Bend


In [5]:
#Exploring member demography dataset
demography.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38747 entries, 0 to 38746
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            38747 non-null  object
 1   createdAt     38747 non-null  object
 2   updatedAt     38747 non-null  object
 3   firstName     38747 non-null  object
 4   lastName      38747 non-null  object
 5   gender        38747 non-null  object
 6   yearOfBirth   38747 non-null  int64 
 7   location      38747 non-null  object
 8   facilityName  38747 non-null  object
dtypes: int64(1), object(8)
memory usage: 2.7+ MB


The member demography table shows that we have 38747 unique members or beneficiaries of our insurance schemes, it has 9 columns and none of them has null/empty values.

In [6]:
#Exploring claims dataset at a glance
claim

,id,createdAt,updatedAt,scheme,patientId,medicationCode,admissionDate,dischargeDate,medicationName,diagnosisCode,procedureCode,quantity,billedAmount,verifiedAmount,paidAmount
0,66dc1204-4659-4a11-9caf-8d73e606f2c8,2024-04-23 00:50:53.250608,2024-04-23 00:50:53.276233,MUTUELLE,ab4dbfa7-9d5d-4341-b3b6-29a4c4b2edf1,32138,2023-10-01 14:30:46.964,2024-03-16 22:54:55.463,HEMORRHOIDECTOMY,ICD-J05AB01003,CPT-42242000FQZ445,75,2625,1716,909
1,71bde428-ef0a-4d42-bff1-473b751fb40e,2024-04-23 00:52:19.464756,2024-04-23 00:52:19.479903,MUTUELLE,5e11bb86-bdf5-460e-96f5-ac1ab31f3bd1,J01MA03001,2024-03-25 20:37:47.191,2024-04-12 20:39:00.497,CIPROFLOXACIN 500MG TABLET,ICD-J05AR01002,CPT-42242000FQZ429,73,521585,222971,298614
2,53ed2514-fad6-4925-abd2-605e272277e7,2024-04-23 00:50:53.317807,2024-04-23 00:50:53.348851,MUTUELLE,d1a7c6dc-cab5-4700-92b7-b406fddbdca4,M01AB05005,2023-08-13 22:05:54.853,2023-12-23 02:15:25.413,DICLOFENAC 75MG/ML INJECTION,ICD-J01XX04001,CPT-42242000FQZ419,100,559500,294490,265010
3,ce6f97e0-7179-4f13-bd59-2e91cbd53442,2024-04-23 00:50:53.377835,2024-04-23 00:50:53.406237,MUTUELLE,f095637f-2192-4a71-aa37-2f3cdbdd3790,A03F01001,2024-03-21 11:19:42.498,2024-04-16 09:17:25.318,METOCLOPRAMIDE 10MG TABLET,ICD-J01XC01001,CPT-42242000FQZ386,86,672520,672469,51
4,a3f82d42-a7f8-4421-bf11-35a53b994fde,2024-04-23 00:51:19.294958,2024-04-23 00:51:19.306367,RAMA,2f5014a3-b453-4ecf-adfb-fdd00292b90a,37438,2023-11-27 21:14:51.064,2024-01-28 18:58:20.373,SCROTUM EXCISION,ICD-J04AM05002,CPT-42242000FQZ419,50,303500,54816,248684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1937431,5162568e-327e-41df-af80-646aba5d33e7,2024-04-23 00:50:52.826295,2024-04-23 00:50:52.843636,RAMA,2e3e82a0-8030-4652-aad5-3a9a80e5fdf4,M01AB05005,2023-06-14 12:22:37.619,2023-07-15 19:59:22.491,DICLOFENAC 75MG/ML INJECTION,ICD-J06BB05001,CPT-42242000FQZ389,64,460800,445741,15059
1937432,c2d16b86-a7f7-428a-8a69-f40121c66377,2024-04-23 00:52:19.3947,2024-04-23 00:52:19.406334,MUTUELLE,c441e5b2-5f8c-4f14-b934-70a48fc6237c,A03F01001,2023-08-16 08:47:50.666,2024-03-27 16:10:32.324,METOCLOPRAMIDE 10MG TABLET,ICD-J04AM05002,CPT-42242000FQZ406,7,10395,3668,6727
1937433,8216f1ae-a8ce-49f4-9d80-edeb68a6085f,2024-04-23 00:52:19.440242,2024-04-23 00:52:19.449886,RAMA,11c238f8-9057-4d61-ab22-ad45c28c5db3,37212,2023-08-15 17:07:29.314,2023-08-25 06:54:32.729,DRAINAGE OF PROSTATIC ABSCESS [PERINEAL],ICD-J05AR05002,CPT-42242000FQZ417,81,526500,239213,287287
1937434,3042e459-1348-4b73-b5b8-181a8834ad50,2024-04-23 00:50:53.154769,2024-04-23 00:50:53.161581,RAMA,05493993-c8c0-446e-9c89-45c65a3e7d51,42131707BWG001,2023-12-05 06:16:16.106,2024-01-09 22:29:55.274,FACE MASK DISPOSABLE,ICD-J06BB04001,CPT-42242000FQZ360,62,267220,15390,251830


In [7]:
#Exploring claims dataset
claim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1937436 entries, 0 to 1937435
Data columns (total 15 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   id              object
 1   createdAt       object
 2   updatedAt       object
 3   scheme          object
 4   patientId       object
 5   medicationCode  object
 6   admissionDate   object
 7   dischargeDate   object
 8   medicationName  object
 9   diagnosisCode   object
 10  procedureCode   object
 11  quantity        int64 
 12  billedAmount    int64 
 13  verifiedAmount  int64 
 14  paidAmount      int64 
dtypes: int64(4), object(11)
memory usage: 221.7+ MB


Claims table shows that we have recieved 1937436 claims in total, 14 columns ranging from claim id, patientId and the amount we paid to the claiming facility

In [8]:
#Verifying if claim has complete information, idealy to identify if all claims received are associated with our member beneficiaries denoted by patientId

incomplete_claims = claim['patientId'].isnull().sum()

if incomplete_claims > 0:
    print(f"There are {incomplete_claims} Claims missing member/patient ID in claims dataset")
else:
    print("There are no Claims missing member/patient ID")

There are 86 Claims missing member/patient ID in claims dataset


Claims data shows that out of 1937436 claims, 86 of them are incomplete claims without reference of our member involved in that claims. i would suggest that we verify this incomplete claims and track missing info.

# Merging member demography and claims

In [9]:
# Rename the 'id' column to 'member_id' in the demography DataFrame
demography = demography.rename(columns={'id': 'member_id'})

# Define columns to keep from demography
columns_to_keep = ['member_id','firstName', 'lastName', 'gender', 'yearOfBirth', 'location', 'facilityName']

# Merge claim_df with demography_df using patientId in claim_df and id in demography_df
demography_claim = pd.merge(claim, demography[columns_to_keep], left_on='patientId', right_on='member_id', how='left')
# Drop the redundant column "member_id" since its the same as patient_id
demography_claim.drop(columns=['member_id'], inplace=True)

In [10]:
#claims_demography at a glance
demography_claim

,id,createdAt,updatedAt,scheme,patientId,medicationCode,admissionDate,dischargeDate,medicationName,diagnosisCode,...,quantity,billedAmount,verifiedAmount,paidAmount,firstName,lastName,gender,yearOfBirth,location,facilityName
0,66dc1204-4659-4a11-9caf-8d73e606f2c8,2024-04-23 00:50:53.250608,2024-04-23 00:50:53.276233,MUTUELLE,ab4dbfa7-9d5d-4341-b3b6-29a4c4b2edf1,32138,2023-10-01 14:30:46.964,2024-03-16 22:54:55.463,HEMORRHOIDECTOMY,ICD-J05AB01003,...,75,2625,1716,909,Alfred,Renner,FEMALE,1997.0,Everett,Chladeda
1,71bde428-ef0a-4d42-bff1-473b751fb40e,2024-04-23 00:52:19.464756,2024-04-23 00:52:19.479903,MUTUELLE,5e11bb86-bdf5-460e-96f5-ac1ab31f3bd1,J01MA03001,2024-03-25 20:37:47.191,2024-04-12 20:39:00.497,CIPROFLOXACIN 500MG TABLET,ICD-J05AR01002,...,73,521585,222971,298614,Thomas,Mosciski,FEMALE,2002.0,Guststad,Brixnessrna
2,53ed2514-fad6-4925-abd2-605e272277e7,2024-04-23 00:50:53.317807,2024-04-23 00:50:53.348851,MUTUELLE,d1a7c6dc-cab5-4700-92b7-b406fddbdca4,M01AB05005,2023-08-13 22:05:54.853,2023-12-23 02:15:25.413,DICLOFENAC 75MG/ML INJECTION,ICD-J01XX04001,...,100,559500,294490,265010,Lana,Huel,FEMALE,1954.0,Cheyanneburgh,Ngemath
3,ce6f97e0-7179-4f13-bd59-2e91cbd53442,2024-04-23 00:50:53.377835,2024-04-23 00:50:53.406237,MUTUELLE,f095637f-2192-4a71-aa37-2f3cdbdd3790,A03F01001,2024-03-21 11:19:42.498,2024-04-16 09:17:25.318,METOCLOPRAMIDE 10MG TABLET,ICD-J01XC01001,...,86,672520,672469,51,Georgia,Grimes,FEMALE,1903.0,Faheychester,Cape Lansgo
4,a3f82d42-a7f8-4421-bf11-35a53b994fde,2024-04-23 00:51:19.294958,2024-04-23 00:51:19.306367,RAMA,2f5014a3-b453-4ecf-adfb-fdd00292b90a,37438,2023-11-27 21:14:51.064,2024-01-28 18:58:20.373,SCROTUM EXCISION,ICD-J04AM05002,...,50,303500,54816,248684,Jacob,D'Amore,FEMALE,1952.0,Providence,La Stamal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1937431,5162568e-327e-41df-af80-646aba5d33e7,2024-04-23 00:50:52.826295,2024-04-23 00:50:52.843636,RAMA,2e3e82a0-8030-4652-aad5-3a9a80e5fdf4,M01AB05005,2023-06-14 12:22:37.619,2023-07-15 19:59:22.491,DICLOFENAC 75MG/ML INJECTION,ICD-J06BB05001,...,64,460800,445741,15059,Tanya,Pfannerstill,FEMALE,1918.0,Oxnard,Spenskialey
1937432,c2d16b86-a7f7-428a-8a69-f40121c66377,2024-04-23 00:52:19.3947,2024-04-23 00:52:19.406334,MUTUELLE,c441e5b2-5f8c-4f14-b934-70a48fc6237c,A03F01001,2023-08-16 08:47:50.666,2024-03-27 16:10:32.324,METOCLOPRAMIDE 10MG TABLET,ICD-J04AM05002,...,7,10395,3668,6727,Julia,Corwin-Harvey,MALE,1909.0,Sipesville,Las Brobroadneck
1937433,8216f1ae-a8ce-49f4-9d80-edeb68a6085f,2024-04-23 00:52:19.440242,2024-04-23 00:52:19.449886,RAMA,11c238f8-9057-4d61-ab22-ad45c28c5db3,37212,2023-08-15 17:07:29.314,2023-08-25 06:54:32.729,DRAINAGE OF PROSTATIC ABSCESS [PERINEAL],ICD-J05AR05002,...,81,526500,239213,287287,Stewart,Greenfelder,FEMALE,1957.0,North Krystinastad,Cape Lansgo
1937434,3042e459-1348-4b73-b5b8-181a8834ad50,2024-04-23 00:50:53.154769,2024-04-23 00:50:53.161581,RAMA,05493993-c8c0-446e-9c89-45c65a3e7d51,42131707BWG001,2023-12-05 06:16:16.106,2024-01-09 22:29:55.274,FACE MASK DISPOSABLE,ICD-J06BB04001,...,62,267220,15390,251830,Marcia,Von,MALE,2009.0,New Vicentaworth,Bea


# Converting from dates of object type to datetime/real date/time datatype

In [11]:
# Convert 'createdAt' column to datetime type with error handling
demography_claim['createdAt'] = pd.to_datetime(demography_claim['createdAt'], errors='coerce')
demography_claim['admissionDate'] = pd.to_datetime(demography_claim['admissionDate'], errors='coerce')

# Extract date component
demography_claim['createdAt_date'] = demography_claim['createdAt'].dt.date
demography_claim['admissionDate'] = demography_claim['admissionDate'].dt.date

Created at data type changes to datetime now

In [12]:
# Filter out rows with missing dates (NaN values)
demography_claim_filtered = demography_claim.dropna(subset=['createdAt_date'])

# Calculate minimum and maximum dates after filtering
if not demography_claim_filtered.empty:
    min_date = demography_claim_filtered['createdAt_date'].min()
    max_date = demography_claim_filtered['createdAt_date'].max()

    print(f"Minimum Date: {min_date}")
    print(f"Maximum Date: {max_date}")
else:
    print("No valid dates found after filtering NaN values.")

Minimum Date: 2024-04-23
Maximum Date: 2024-04-23


This analysis shows that all claims were made on 23rd april 2024

In [13]:
# Convert 'DOB' column to datetime format
#demography_claim['yearOfBirth'] = pd.to_datetime(demography_claim['yearOfBirth'], errors='coerce')

# Calculate age based on date of birth (DOB)
demography_claim['AgePerDOB'] = 2024 - demography_claim['yearOfBirth']

# Handle the case where DOB is missing
demography_claim.loc[demography_claim['yearOfBirth'].isnull(), 'AgePerDOB'] = pd.NA

In [14]:
# Define a function to categorize age into age groups
def categorize_age(age):
    if pd.isna(age):
        return "Not specified"
    elif age < 15:
        return "Under 15"
    elif 15 <= age < 25:
        return "15-25"
    elif 25 <= age < 55:
        return "25-55"
    elif 55 <= age <= 65:
        return "55-65"
    else:
        return "Above 65"

# Apply the function to create the new column AgeGroup
demography_claim['AgeGroup'] = demography_claim['AgePerDOB'].apply(categorize_age)

In [15]:
#Filtering only important columns necesary for analysis
demography_claim=demography_claim[['id', 'createdAt', 'updatedAt', 'scheme', 'patientId', 'firstName', 'lastName', 'gender', 'yearOfBirth','AgeGroup'
       ,'location', 'facilityName','medicationCode','admissionDate', 'dischargeDate', 'medicationName', 'diagnosisCode',
       'procedureCode', 'quantity', 'billedAmount', 'verifiedAmount','paidAmount']]

# Analytics

In [16]:
# Calculate the total number of unique claims based on the 'id' column
total_claims = len(demography_claim['id'].unique())
print(f"All claims received: {total_claims}")

All claims received: 1937436


In [17]:
# Filter claims where 'patientId' is null
missing_patientId_claims = demography_claim[demography_claim['patientId'].isnull()]

# Calculate the total number of claims with missing 'patientId'
total_missing_patientId_claims = len(missing_patientId_claims)

print(f"Total Claims Received: {total_claims}")
print(f"Claims with Missing 'patientId': {total_missing_patientId_claims}")
print(f"Claims with complete info: {total_claims-total_missing_patientId_claims}")

Total Claims Received: 1937436
Claims with Missing 'patientId': 86
Claims with complete info: 1937350


In [18]:
missing_patientId_claims

,id,createdAt,updatedAt,scheme,patientId,firstName,lastName,gender,yearOfBirth,AgeGroup,...,medicationCode,admissionDate,dischargeDate,medicationName,diagnosisCode,procedureCode,quantity,billedAmount,verifiedAmount,paidAmount
204068,06432e0b-e610-4f04-9633-1071705d0f75,2024-04-23 00:53:26.732837,2024-04-23 00:53:26.732837,RAMA,NaN,NaN,NaN,NaN,NaN,Not specified,...,P01AB01001,2023-08-12,2023-11-16 17:28:57.933,METRONIDAZOLE 250MG TABLET,ICD-J04BA02001,CPT-42242000FQZ394,90,442350,290507,151843
279476,62c1f676-4c16-4cc0-9b62-1e3733f71806,2024-04-23 00:53:26.720794,2024-04-23 00:53:26.720794,RAMA,NaN,NaN,NaN,NaN,NaN,Not specified,...,35539,2023-07-22,2024-03-27 03:27:22.27,CAUTERIZATION OF EXTENSIVE VULVAR CONDYLOMA (...,ICD-J05AG03002,CPT-42242000FQZ428,23,27140,20956,6184
279477,f2821d37-1969-45d5-8160-1b94447f3088,2024-04-23 00:53:26.721235,2024-04-23 00:53:26.721235,MUTUELLE,NaN,NaN,NaN,NaN,NaN,Not specified,...,30229,2023-09-09,2024-03-23 10:15:23.227,"FASCIOTOMY (MUSCLE, EXCISION OF [EXTENSIVE])",ICD-J04AA01002,CPT-42242000FQZ395,86,540080,133511,406569
279478,4b8b80ed-fba2-4234-b27e-77e73abfd294,2024-04-23 00:53:26.721585,2024-04-23 00:53:26.721585,MUTUELLE,NaN,NaN,NaN,NaN,NaN,Not specified,...,20960,2024-01-07,2024-01-21 13:22:19.434,INITIATION & MANAGEMENT OF ANAESTHESIA FOR PE...,ICD-J06BB05001,CPT-42242000FQZ410,41,409180,383220,25960
279479,1a6a35f8-58d3-4537-8e4c-1eb3043e6472,2024-04-23 00:53:26.721930,2024-04-23 00:53:26.72193,RAMA,NaN,NaN,NaN,NaN,NaN,Not specified,...,32147-1,2023-08-28,2024-02-15 07:41:46.148,HEMORRHOIDECTOMY (PERIANAL THROMBOSIS),ICD-J02AC01001,CPT-42242000FQZ406,85,1700,484,1216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1937419,c9551c1b-7468-4c82-b692-9a21b6cb01c2,2024-04-23 01:11:55.354950,2024-04-23 01:11:55.35495,RAMA,NaN,NaN,NaN,NaN,NaN,Not specified,...,P03AX01004,2024-01-22,2024-02-04 17:00:29.432,"BENZYL BENZOATE12,5%METHANOL 100ML",ICD-J05AP08001,CPT-42242000FQZ355,84,629160,339950,289210
1937420,454813b1-120e-4362-ad4c-e46c0850dac8,2024-04-23 01:11:55.355249,2024-04-23 01:11:55.355249,MUTUELLE,NaN,NaN,NaN,NaN,NaN,Not specified,...,42131707BWG004,2023-04-01,2023-11-10 10:35:03.143,FACE MASK SIZE 1,ICD-J05AG01001,CPT-42242000FQZ411,49,395920,34475,361445
1937421,3a22fce4-5592-4f2c-8cb1-6707c94dae67,2024-04-23 01:11:55.355552,2024-04-23 01:11:55.355552,RAMA,NaN,NaN,NaN,NaN,NaN,Not specified,...,20900,2024-02-16,2024-04-16 20:13:07.929,INITIATION & MANAGEMENT OF ANAESTHESIA FOR PE...,ICD-J05AE08002,CPT-42242000FQZ427,45,436050,49975,386075
1937422,662a90a7-5f97-425e-a823-ee33edc26fc1,2024-04-23 01:11:55.355962,2024-04-23 01:11:55.355962,RAMA,NaN,NaN,NaN,NaN,NaN,Not specified,...,32000,2023-06-14,2024-03-17 13:54:34.161,LARGE INTESTINE RESECTION (WITHOUT ANASTOMOSIS),ICD-J05AB01003,CPT-42242000FQZ385,98,258230,239977,18253


all these 86 claims have missing data

In [19]:
# removing claims with missing data
demography_claim_complete = demography_claim.dropna(subset=['patientId'])

# Task 1: Identifying High-Cost Diagnoses and Procedures

In [20]:
# Calculate total billed amount for each diagnosis code
diagnosis_costs = demography_claim_complete.groupby('diagnosisCode')['billedAmount'].sum().reset_index()
diagnosis_costs = diagnosis_costs.sort_values(by='billedAmount', ascending=False)

# Calculate total billed amount for each procedure code
procedure_costs = demography_claim_complete.groupby('procedureCode')['billedAmount'].sum().reset_index()
procedure_costs = procedure_costs.sort_values(by='billedAmount', ascending=False)

In [21]:
# Create a PrettyTable for diagnoses
diagnosis_table = PrettyTable()
diagnosis_table.field_names = ['Diagnosis Code', 'Billed Amount']

# Add rows to the table for top 5 high-cost diagnoses
for index, row in diagnosis_costs.head(5).iterrows():
    diagnosis_table.add_row([row['diagnosisCode'], row['billedAmount']])

# Create a PrettyTable for procedures
procedure_table = PrettyTable()
procedure_table.field_names = ['Procedure Code', 'Billed Amount']

# Add rows to the table for top 5 high-cost procedures
for index, row in procedure_costs.head(5).iterrows():
    procedure_table.add_row([row['procedureCode'], row['billedAmount']])

# Print the tables
print("Top 5 High-Cost Diagnoses:")
print(diagnosis_table)

print("\nTop 5 High-Cost Procedures:")
print(procedure_table)

Top 5 High-Cost Diagnoses:
+----------------+---------------+
| Diagnosis Code | Billed Amount |
+----------------+---------------+
| ICD-J06AA01002 |   5111752850  |
| ICD-J04BA02002 |   5096369155  |
| ICD-J05AG03002 |   5093472260  |
| ICD-J07BC01002 |   5089160145  |
| ICD-J05AR02001 |   5088210710  |
+----------------+---------------+

Top 5 High-Cost Procedures:
+--------------------+---------------+
|   Procedure Code   | Billed Amount |
+--------------------+---------------+
| CPT-42242000FQZ435 |   5020133075  |
| CPT-42242000FQZ380 |   5016557745  |
| CPT-42242000FQZ406 |   5002706570  |
| CPT-42242000FQZ445 |   4990459705  |
| CPT-42242000FQZ398 |   4989801765  |
+--------------------+---------------+


# Task 2: Analyzing Variations in Costs

In [22]:
# Variation based on member demographics
# Gender demographic
gender_demographic_variation = demography_claim_complete.groupby('gender')['billedAmount'].mean().round(0)
print("\nMean Billed Amount by Gender:")
print(gender_demographic_variation)


Mean Billed Amount by Gender:
gender
FEMALE    252613.0
MALE      252625.0
Name: billedAmount, dtype: float64


In [23]:
#female_claim_average=round(gender_demographic_variation.loc['FEMALE'])
#male_claim_average=round(gender_demographic_variation.loc['MALE'])

The analysis indicates minimal variation in healthcare costs between genders, suggesting that gender-based distinctions may not significantly influence overall cost reduction strategies within the medical claims dataset. Notably, the slightly higher average billing for men could be attributed to potentially higher-cost medications or treatments, despite women's more frequent healthcare facility visits. This insight underscores the complexity of cost determinants and highlights the need for comprehensive analysis beyond gender demographics to effectively target cost-saving initiatives.

In [24]:
# # Variation based on age group demographic
age_variation = demography_claim_complete.groupby('AgeGroup')['billedAmount'].mean().round(0)
age_variation= age_variation.sort_values(ascending=False)
print("\nMean Billed Amount by age group demographic:")
print(age_variation)


Mean Billed Amount by age group demographic:
AgeGroup
55-65       252885.0
15-25       252810.0
Above 65    252741.0
25-55       252533.0
Under 15    252360.0
Name: billedAmount, dtype: float64


The analysis of mean billed amounts by age group demographic shows a relatively consistent pattern across different age categories. Despite minor variations, ranging from 252,360.0 for the "Under 15" age group to 252,885.0 for the "55-65" age group, the differences in billed amounts appear to be minimal. This suggests that age group demographics may not be a prominent factor influencing healthcare costs within the dataset, emphasizing the need to explore other variables for targeted cost reduction strategies.

In [30]:
# # Variation based on age group demographic
Health_Facility_variation = demography_claim_complete.groupby('facilityName')['billedAmount'].mean().round(0)
Health_Facility_variation= Health_Facility_variation.sort_values(ascending=False)
print("\nMean Billed Amount by health facility demographic:")

# Convert the Series to a DataFrame
df = Health_Facility_variation.reset_index()
df.columns = ['facilityName', 'meanBilledAmount']
df


Mean Billed Amount by health facility demographic:


,facilityName,meanBilledAmount
0,North Clairesit Du Grayscloud,257190.0
1,Muchstur Under Asperc Haven,256372.0
2,El Gelrnie,255864.0
3,Mount Squimond,255698.0
4,Mat Haven,255635.0
...,...,...
95,Whiteyarm,249867.0
96,New Gessdon,249800.0
97,Battelklouspa,249707.0
98,Brixnessrna,249674.0


In [31]:
# Save the DataFrame back to a new Excel file or overwrite the existing one for further analysis in excel
df.to_excel('Mean_Billed_Amount_by_health_facility_demographic.xlsx', index=False)

The analysis of mean billed amounts by health facility demographic reveals varying costs across different healthcare facilities. health facilities such as North Clairesit Du Grayscloud and Muchstur Under Asperc Haven exhibit higher mean billed amounts compared to other facilities like Ntorers Bay and Brixnessrna. These findings suggest notable differences in healthcare costs based on facility locations within the dataset, highlighting the importance of facility-specific cost management strategies to optimize our costs effectively.

In [26]:
# # Variation based on plan type
plan_variation = demography_claim_complete.groupby('scheme')['billedAmount'].mean().round(0)
print("\nMean Billed Amount by Plan Type:")
print(plan_variation)


Mean Billed Amount by Plan Type:
scheme
MUTUELLE    252943.0
RAMA        252294.0
Name: billedAmount, dtype: float64


The analysis reveals variations in billed amounts based on member plan types. Specifically, individuals under the MUTUELLE scheme exhibit a slightly higher mean billed amount compared to those under the RAMA scheme, highlighting potential differences in healthcare costs associated with different insurance plans within the dataset. Understanding these variations can inform targeted cost reduction strategies tailored to specific plan types to optimize healthcare expenditure management.

In [33]:
# # Variation based on geographic location
location_variation = demography_claim_complete.groupby('location')['billedAmount'].mean().round(0)
location_variation = location_variation.sort_values(ascending=False)
print("\nMean Billed Amount by Geographic Location:")

# Convert the Series to a DataFrame
df = location_variation.reset_index()
df.columns = ['location', 'meanBilledAmount']
df


Mean Billed Amount by Geographic Location:


,location,meanBilledAmount
0,New Braxtonton,379238.0
1,New Cordellfield,377660.0
2,Lucindachester,377595.0
3,East Rey,377138.0
4,Hicklecester,373164.0
...,...,...
28063,North Emeraldhaven,151851.0
28064,Camrynborough,145440.0
28065,Fort Cristal,142872.0
28066,North Oswaldview,134803.0


In [34]:
# Save the DataFrame back to a new Excel file or overwrite the existing one for further analysis in excel
df.to_excel('Mean_Billed_Amount_by_location.xlsx', index=False)

The analysis of mean billed amounts by geographic location reveals substantial variations in healthcare costs across different regions. Locations such as New Braxtonton and New Cordellfield exhibit notably higher mean billed amounts compared to other areas like Port Keyonville and North Oswaldview. These findings highlight geographical disparities in healthcare expenditure within the dataset, suggesting the importance of regional-specific cost management strategies to optimize healthcare budgeting and resource allocation effectively.

# 3. Can you identify any potential areas for cost savings through negotiations with healthcare providers or by implementing disease management programs?

Based on the analysis of our medical claims dataset for RSSB insurance members, I have identified several key areas where cost savings can be achieved through strategic negotiations with healthcare providers and targeted disease management programs:

Identifying High-Cost Diagnoses and Procedures: By focusing on negotiating lower costs or implementing efficient treatment protocols for high-cost diagnoses and procedures identified in the dataset (e.g., ICD codes and procedure codes), we can optimize expenditures while maintaining quality care.

Geographic Cost Management: Targeting areas with notably high mean billed amounts by geographic location (e.g., New Braxtonton, New Cordellfield) enables us to implement region-specific cost-saving initiatives or negotiate favorable provider contracts, enhancing efficiency and resource allocation.

Health Facility Optimization: Implementing strategies to negotiate costs or optimize care delivery at facilities with higher mean billed amounts (e.g., North Clairesit Du Grayscloud, Muchstur Under Asperc Haven) can significantly contribute to cost efficiencies and improved financial performance.

Age Group and Gender-Neutral Programs: Developing universal disease management programs that benefit all age groups and genders, considering the minimal variations observed in billed amounts, ensures equitable and effective healthcare interventions without significant cost disparities.
These insights provide concrete avenues for RSSB to engage in proactive negotiations with healthcare providers and implement targeted disease management initiatives aimed at optimizing healthcare expenditure while upholding the quality of care delivered to our insurance members. By leveraging these strategic opportunities, RSSB can achieve sustainable cost savings and enhance the overall value proposition for its members and stakeholders.